# パッケージ

In [1]:
import numpy as np
import pandas as pd

# 事前準備  
時系列株価データの取得

In [2]:
from datetime import date, timedelta
from pandas_datareader import data as pdr

In [3]:
# 期間設定
today = date.today()
startday = today - timedelta(days=20)
# 銘柄設定
ticker_list = ["IBM", "AAPL", "MSFT", "C", "T"]

In [4]:
# 取得
pd_data = pdr.DataReader(ticker_list, "stooq", startday)

In [5]:
# リターン
ret = pd_data['Close'].sort_index(ascending=True)
ret=ret.pct_change().drop(ret.index[0])

In [49]:
# 共分散行列
cvm = ret.cov()

## 2.1 リスク分布RのPCAウェイト  
各主成分のリスク寄与が指定した配分となるような資産ウェイトを算出

In [36]:
# パラメータ設定（共分散行列、リスク水準）
cov = cvm
riskTarget = 1

In [22]:
# 固有値、固有ベクトル（入力はエルミート行列である必要がある⇒下三角行列で固有値を算出するため）
eVal, eVec = np.linalg.eigh(cov)

In [23]:
# 降順ソートのインデックス順列
indices = eVal.argsort()[::-1]

In [24]:
# 固有値降順で並び替え
eVal, eVec = eVal[indices], eVec[:, indices]

In [29]:
# デフォルトのリスク分布を設定（固有値最低の主成分に全て配分）
riskDist = np.zeros(cov.shape[0])
riskDist[-1] = 1

In [37]:
# 設定したリスク分布となる資産ウェイトの算出
loads = riskTarget*riskDist/eVal**.5
wghts = np.dot(eVec, np.reshape(loads, (-1, 1)))

In [9]:
prc = pd_data["Close"]["IBM"].sort_index(ascending=True)

In [12]:
diff = prc.diff()